In [ ]:
pip install transformers torch gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

def generate_poem(keywords):

    prompt = f"Write a beautiful and meaningful 6-line poem using these words: {keywords}."


    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

    #  min_length
    outputs = model.generate(
        inputs['input_ids'],
        min_length=100,
        max_length=250,
        num_beams=5,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        early_stopping=True
    )


    poem = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return poem


if __name__ == "__main__":
    keywords = input("Enter keywords separated by commas: ")
    print("\nGenerated Poem:\n", generate_poem(keywords))



Enter keywords separated by commas: roses, river, trees

Generated Poem:
 roses on the banks of a river with trees in the middle a beautiful and meaningful poem i would like to share with you a poem about the beauty of the river and the trees that surround it i hope you will enjoy this poem as much as i do if you do n't please leave me alone i want you to know that i love you and i will never leave you i promise i have a heart full of love for you and that you will always be in my heart


Rule based model

In [ ]:
import random

def rule_based_poem_generator(keywords):

    words = [word.strip() for word in keywords.split(",")]


    templates = [
        "The {0} flows under the {1} sky,",
        "Among the {1}, the winds fly high.",
        "Whispers rise from the {2} deep,",
        "Nature awakens from its sleep.",
        "The {0} sings a lullaby,",
        "While {2} dreams are drifting by."
    ]


    random.shuffle(words)


    while len(words) < 3:
        words.append(random.choice(words))


    poem_lines = [templates[i].format(words[0], words[1], words[2]) for i in range(6)]


    poem = "\n".join(poem_lines)
    return poem


if __name__ == "__main__":
    keywords = input("Enter keywords separated by commas: ")
    print("\nGenerated Poem (Rule-Based):\n")
    print(rule_based_poem_generator(keywords))


Enter keywords separated by commas: trees, flowers, buds

Generated Poem (Rule-Based):

The buds flows under the flowers sky,
Among the flowers, the winds fly high.
Whispers rise from the trees deep,
Nature awakens from its sleep.
The buds sings a lullaby,
While trees dreams are drifting by.


Statistical model

In [ ]:
import random
import re
from collections import defaultdict


training_corpus = """
The river flows through the forest,
The trees whisper to the sky,
The forest hides secrets deep,
The sun kisses the river’s face,
The breeze dances with the trees,
Nature sings a gentle lullaby.
"""

def build_ngram_model(corpus, n=3):
    model = defaultdict(list)
    tokens = re.findall(r'\b\w+\b', corpus.lower())
    for i in range(len(tokens) - n):
        key = tuple(tokens[i:i + n - 1])
        model[key].append(tokens[i + n - 1])
    return model

def generate_poem(model, num_words=50, words_per_line=7):
    start = random.choice(list(model.keys()))
    output = list(start)

    for _ in range(num_words):
        next_words = model.get(tuple(output[-(len(start)):]), None)
        if not next_words:
            break
        output.append(random.choice(next_words))


    poem_lines = []
    for i in range(0, len(output), words_per_line):
        line = ' '.join(output[i:i+words_per_line])
        poem_lines.append(line.capitalize())

    return '\n'.join(poem_lines)

if __name__ == "__main__":
    ngram_model = build_ngram_model(training_corpus, n=3)
    poem = generate_poem(ngram_model, num_words=42, words_per_line=7)
    print("\nGenerated Poem (Statistical N-Gram):\n")
    print(poem)



Generated Poem (Statistical N-Gram):

Sun kisses the river flows through the
Forest the trees whisper to the sky
The forest the trees whisper to the
Sky the forest hides secrets deep the
Sun kisses the river s face the
Breeze dances with the trees nature sings
A gentle


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
import random


corpus = """
The river flows through the forest
The trees whisper to the sky
The forest hides secrets deep
The sun kisses the river’s face
The breeze dances with the trees
Nature sings a gentle lullaby
"""


tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus])
total_words = len(tokenizer.word_index) + 1


input_sequences = []
words = corpus.lower().split()
for i in range(2, len(words)):
    seq = words[i-2:i+1]
    encoded_seq = tokenizer.texts_to_sequences([' '.join(seq)])[0]
    if len(encoded_seq) == 3:
        input_sequences.append(encoded_seq)


input_sequences = np.array(input_sequences)
X = input_sequences[:, :-1]
y = to_categorical(input_sequences[:, -1], num_classes=total_words)


model = Sequential()
model.add(Embedding(total_words, 10, input_length=2))
model.add(LSTM(150, return_sequences=False))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X, y, epochs=500, verbose=0)


def generate_poem(seed_text, next_words=30):
    result = []
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0][-2:]
        token_list = np.array(token_list).reshape(1, 2)
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted)
        output_word = ''
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                output_word = word
                break
        if output_word == '':
            break
        seed_text += ' ' + output_word
        result.append(output_word)
    return ' '.join(result)


seed_text = "The river"
generated_poem = generate_poem(seed_text)


poem_lines = []
words = generated_poem.split()
for i in range(0, len(words), 6):
    poem_lines.append(' '.join(words[i:i+6]).capitalize())

print("\nGenerated Poem (RNN LSTM Model):\n")
print('\n'.join(poem_lines))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Generated Poem (RNN LSTM Model):

Flows through the forest the trees
Nature sings a gentle lullaby through
The forest the trees nature sings
A gentle lullaby through the forest
The trees nature sings a gentle


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set the seed text (keywords or theme)
prompt = "The river flows through the forest and the"

# Encode the input
inputs = tokenizer.encode(prompt, return_tensors="pt")

# Generate text (auto-regressive generation)
outputs = model.generate(
    inputs,
    max_length=100,         # Total length (prompt + generated)
    num_return_sequences=1, # Only one output
    no_repeat_ngram_size=2, # Avoid repeating phrases
    temperature=0.7,        # Controls randomness
    top_p=0.9,              # Nucleus sampling
    do_sample=True          # Sampling instead of greedy
)

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\nGenerated Output (Auto-Regressive GPT-2):\n")
print(generated_text)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Generated Output (Auto-Regressive GPT-2):

The river flows through the forest and the river is a great river.

When the rivers pass through a city, they are called rivers. The river passes through all the cities and is called the city. But when a river runs through an area, it is not called a River. It is the name of the place that the water runs in. And the town that runs into a village is known as a town. When a person is born, his father dies and his mother dies. This


In [ ]:
import language_tool_python

def grammar_check(text):
    tool = language_tool_python.LanguageTool('en-US')
    matches = tool.check(text)
    corrected = language_tool_python.utils.correct(text, matches)
    return corrected

poem = "The tree sings beautifully in sky\nThe leaves falls and fly\n"
print("Before Grammar Correction:\n", poem)
print("After Grammar Correction:\n", grammar_check(poem))


ModuleNotFoundError: No module named 'language_tool_python'

In [ ]:
pip install language_tool_python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.9 MB/s eta 0:00:00


In [ ]:
prompt = "Write a joyful and uplifting poem about nature:\n"


In [ ]:
import language_tool_python

def grammar_check(text):
    tool = language_tool_python.LanguageTool('en-US')
    matches = tool.check(text)
    corrected = language_tool_python.utils.correct(text, matches)
    return corrected

poem = "The tree sings beautifully in sky\nThe leaves falls and fly\n"
print("Before Grammar Correction:\n", poem)
print("After Grammar Correction:\n", grammar_check(poem))


Before Grammar Correction:
 The tree sings beautifully in sky
The leaves falls and fly



INFO:language_tool_python.download_lt:Unzipping /tmp/tmpinije1nk.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.5.zip to /root/.cache/language_tool_python.


After Grammar Correction:
 The tree sings beautifully in sky
The leaves falls and fly

